[The resource](https://nbviewer.jupyter.org/github/jamesdellinger/fastai_deep_learning_course_part2_v3/blob/master/02_fully_connected_my_reimplementation.ipynb?flush_cache=true)
# Forward & Backward Passes from Foundations

#### Last Time
In the [previous notebook](http://nbviewer.jupyter.org/github/jamesdellinger/fastai_deep_learning_course_part2_v3/blob/master/01_matmul_my_reimplementation.ipynb?flush_cache=true), we compared and contrasted techniques for implementing matrix multiplication from scratch. We saw that computation time decreased as we moved from using pure Python for-loops to broadcasting, and then we sped things up even further by employing [einstein summation](https://docs.scipy.org/doc/numpy/reference/generated/numpy.einsum.html).

Finally, and admittedly somewhat disappointingly, we confirmed that using the built-in PyTorch matmul method offered the fastest performance, thanks to the fact that it hands the operation off to the Nividia [BLAS](https://docs.nvidia.com/cuda/index.html), whose low-level routines have been optimized to perform linear algebraic calculations as rapidly as possible on Nvidia GPUs. The somewhat disappointing aspect of this is that we, as developers, don't have any vision into exactly what's going on inside Nvidia's subroutines. We have to take it at its word that it's doing things the best possible way, and while its speed would seem to indicate that this is the case, it still would be nice to be able to write our code in the language that the compiler will be interpreting.

#### Forward and Backward Passes
In this notebook we follow the same pattern as the last and experiment with various approaches to implementing forward and back propagation for a fully connected neural network. We'll continue to use the [MNIST](https://en.wikipedia.org/wiki/MNIST_database) dataset to observe how our code performs.

#### Weight Initialization
During the course of this notebook, I also embark on an exploration of the reasons why properly initializing neural network weight layers is so important. Much of my work is inspired by a [notebook](https://github.com/fastai/fastai_docs/blob/master/dev_course/dl2/02b_initializing.ipynb) that was covered during the [2nd lesson (aka "Lesson 9")](https://youtu.be/AcA8HAYh7IE) lecture of the 2019 fast.ai Deep Learning part II course.

I show a series of simple experiments that illustrate the hows and whys of weight init, while touching on previous work done by [Glorot and Bengio](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf), and by [He, et. al.](https://arxiv.org/pdf/1502.01852.pdf) Much of my work in this notebook served as a basis for a [medium post](https://towardsdatascience.com/weight-initialization-in-neural-networks-a-journey-from-the-basics-to-kaiming-954fb9b47c79?source=---------2------------------) I wrote on the topic of weight init.

The [2nd lesson ("Lesson 9")](https://youtu.be/AcA8HAYh7IE) also covered [this notebook](https://github.com/fastai/fastai_docs/blob/master/dev_course/dl2/02a_why_sqrt5.ipynb) that explored the history behind the PyTorch library's use of $\sqrt{5}$, instead of the Kaiming paper's recommended value of $\sqrt{2}$ when generating a Kaiming weight initialization. I omit this side bar from my discussion in this notebook, but suffice to say, we learned in our lecture that the use of $\sqrt{5}$ was due to a long-standing bug that members of the PyTorch team throught worked better than the paper's default of $\sqrt{2}$. [Jeremy's notebook](https://github.com/fastai/fastai_docs/blob/master/dev_course/dl2/02a_why_sqrt5.ipynb) demonstrated that this was not quite the case, and the PyTorch team has recently revised the algorithm for calculating Kaiming, removing the $\sqrt{5}$.

#### Attribution
Virtually all the code that appears in this notebook is the creation of [Sylvain Gugger](https://www.fast.ai/about/#sylvain) and [Jeremy Howard](https://www.fast.ai/about/#jeremy). The original versions of the notebooks that they made for the course lecture, from which I drew the material that appears below, can be found [here](https://github.com/fastai/course-v3/blob/master/nbs/dl2/02_fully_connected.ipynb) and [here](https://github.com/fastai/course-v3/blob/master/nbs/dl2/02b_initializing.ipynb). I simply re-typed, line-by-line, the pieces of logic necessary to implement the functionality that their notebooks demonstrated. In some cases I changed the order of code cells and or variable names so as to fit an organization and style that seemed more intuitive to me. Any and all mistakes are my own.

On the other hand, all long-form text explanations in this notebook are solely my own creation. Writing extensive descriptions of the concepts and code in plain and simple English forces me to make sure that I actually understand how they work.

(https://towardsdatascience.com/weight-initialization-in-neural-networks-a-journey-from-the-basics-to-kaiming-954fb9b47c79)

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
#export

from exports.nb_01 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train, y_train, x_valid, y_valid)) # convert train/val to torch tensors

def normalize(x, m, s): return (x-m)/s

In [ ]:
x_train, y_train, x_valid, y_valid = get_data()

Before we train a neural network on our MNIST images, we first need to modify the floating point values inside the tensors that contain each image, such that across all the values of all the images in the training set, the values fall into a normal distribution.

If we were to skip this step, and the gaps between different values were too skewed, it would take much longer, or possibly be impossible for our network to learn patterns from the MNIST digit handwriting images. More specifically, if two pixels are more similar than different, without proper normalization of their numerical values, it may be highly difficult for the network to understand that the pixels are in fact similar. The same would also be true for learning that two very different pixels are in fact different.

Furthermore, as we will see below, different approaches for network layer weight initialization presume that inputs will fall into a normal distribution.

We can see right below that straight out of the box, our imageset's numerical values don't follow a normal distribution:

In [4]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(0.1304), tensor(0.3073))

In [5]:
x_train = normalize(x_train, x_train.mean(), x_train.std())

In [6]:
x_train.mean(), x_train.std()

(tensor(-3.0466e-06), tensor(1.))

The values inside the images in the training set now all follow a normal distribution, with a mean and standard deviation hovering around 0 and 1, respectively.

We also need to normalize the images in the validation set. The key thing here is to remember that since this is a validation set, we can't allow any information from its images to leak into our model. Therefore, while we do normalize it, we do so by using the mean and standard deviation of our training set.

In [7]:
x_valid.mean(), x_valid.std()

(tensor(0.1287), tensor(0.3050))

In [8]:
# Be sure to use training mean and standard dev to normalize val set!
x_valid = normalize(x_valid, train_mean, train_std)

In [9]:
x_valid.mean(), x_valid.std()

(tensor(-0.0059), tensor(0.9924))

In [10]:
#export
def test_near_zero(a, tol=1e-3): assert a.abs()<tol, f'Near zero: {a}'

In [11]:
test_near_zero(x_train.mean())
test_near_zero(1 - x_train.std())

In [12]:
n,m = x_train.shape
c = y_train.max() + 1
n,m,c

(50000, 784, tensor(10))

## Basic Foundations Version of Forward/Backprop

We'll first write the logic for forward and backward passes using several simple methods. Later on in the notebook, we'll refactor these into better-organized classes.

Our network's hidden layer will contain 50 cells:

In [13]:
nh = 50

### Weight Initialization

The aim of weight initialization is to prevent layer activation outputs from exploding or vanishing during the course of a forward pass through a deep neural network. If either occurs, loss gradients will either be too large or too small to flow backwards beneficially, and networks will take longer to converge, if they are even able to do so at all. 

Let's first think about initial weight values in the context of how neural network layers work. Matrix multiplication is the basic operation in a neural network. Here's a quick-and-dirty thought exercise to help us get a conceptual feel for why weight initialization matters in neural nets with several layers. i.e. neural nets where matrix multiplications are performed over and over in succession.

Let's pretend we have a vector `x` that contains some network inputs, and that the matrix `a` represents a layer's weights.

It is the standard practice when training neural networks to ensure that our inputs' values are scaled such that they fall inside such a normal distribution with a mean of 0 and a standard deviation of 1.

In [14]:
x = torch.randn(512)

Now it turns out that also initializing the values of layer input weights from the same standard normal distribution is never a good idea. To see why this is, let's run through a quick example where we do just that, and see what happens. 

Say we had a simple hundred-layer network with no activations, in order to complete a single forward pass we'd have to perform a single matrix multiplication between layer inputs and weights at each of the hundred layers, for a grand total of *100* consecutive matrix multiplications.

We can simulate this by multiplying `x` by `a` 100 times in a row.

In [15]:
for i in range(100): 
    a = torch.randn(512,512)
    x = a @ x
x.mean(), x.std()

(tensor(nan), tensor(nan))

Whoa! Somewhere during 100 multiplications, the outputs got so big that even the computer wasn't able to recognize their standard deviation and mean as numbers. We can see exactly how long that took to happen.

In [16]:
x = torch.randn(512)

for i in range(100):
    a = torch.randn(512,512)
    x = a @ x
    if torch.isnan(x.std()): break
i

28

The activation outputs exploded within 29 of our hypothetical network "layers." We clearly initialized our weights (the `a` matrix) to be too large.

Now let's see the other side of the coin: what happens when we initialize network weights to be too small -- we'll scale our initial weights values so that, although still being in a normal distribution with a mean about 0, they have a standard deviation of 0.01.

In [17]:
x = torch.randn(512)

for i in range(100): 
    a = torch.randn(512,512) * 0.01
    x = a @ x
x.mean(), x.std()

(tensor(0.), tensor(0.))

During the course of the above hypothetical forward pass, the activation completely vanished. 

To sum it up, if weights are initialized too large, the network won't learn well. The same happens when weights are initialized too small. How can we find the sweet spot?

Remember that as mentioned above, the math required to complete a forward pass through a neural network entails nothing more than a succession of matrix multiplications. If we have an output `y` that is the product of a matrix multiplication between our input vector `x` and weight matrix `a`, each element $i$ in `y` is defined as $$y_{i} = \sum_{k=0}^{n-1}a_{i,k}x_{k}$$
where $i$ is a given row-index of weight matrix `a`, $k$ is both a given column-index in weight matrix `a` and element-index in input vector `x`, and $n$ is the range or total number of elements in `x`. This can also be defined in Python as
```
y[i] = sum([c*d for c,d in zip(a[i], x)])
```
We can demonstrate that the matrix product of our inputs `x` and weight matrix `a` that we initialized from a standard normal distribution will, on average, have a standard deviation very close to the square root of the number of input connections, or $\sqrt{512}$ in our example.

In [18]:
mean, var = 0.,0.
for i in range(10000):
    x = torch.randn(512)
    a = torch.randn(512,512)
    y = a @ x
    mean += y.mean().item()
    var += y.pow(2).mean().item()
mean/10000, math.sqrt(var/10000)

(0.00488822050280869, 22.62951276631407)

In [19]:
math.sqrt(512)

22.627416997969522

This property isn't surprising when we consider it in terms of how matrix multiplication is defined: in order to calculate `y` we sum 512 products of the element-wise multiplication of one element of the inputs `x` by one column of the weights `a`. In our example where both `x` and `a` are initialized using standard normal distributions, each of these 512 products would have a mean of 0 and standard deviation of 1.

In [20]:
mean, var = 0.,0.
for i in range(10000):
    x = torch.randn(1)
    a = torch.randn(1)
    y = a*x
    mean += y.item()
    var += y.pow(2).item()
mean/10000, math.sqrt(var/10000)

(0.0014878797999873313, 1.0013958118334958)

It then follows that the *sum* of these 512 products would have a mean of 0, variance of 512, and therefore a standard deviation of $\sqrt{512}$. 

And this is why in our example above we saw our layer outputs exploding after 29 consecutive matrix multiplications. In the case of our simple 100-layer architecture, what we'd like is for each layer's outputs to have a standard deviation of about 1. This conceivably would allow us to repeat matrix multiplications across as many network layers as we want, without activations exploding or vanishing.

If we first scale the weight matrix `a` by dividing all its randomly chosen values by $\sqrt{512}$, the element-wise multiplication that fills in one element of the outputs `y` would now have a variance of $\frac{1}{512}$.

In [21]:
mean, var = 0.,0.
for i in range(10000):
    x = torch.randn(1)
    a = torch.randn(1)*math.sqrt(1./512)
    y = a*x
    mean += y.item()
    var += y.pow(2).item()
mean/10000, var/10000

(-8.146830289721114e-05, 0.0018881072559876757)

In [22]:
1/512

0.001953125

This means that the standard deviation of the matrix `y`, which contains each of the 512 values that are generated by way of matrix multiplication of inputs `x` and weights `a`, would be 1. Let's confirm this experimentally.

In [23]:
mean, var = 0.,0.
for i in range(10000):
    x = torch.randn(512)
    a = torch.randn(512,512)*math.sqrt(1./512)
    y = a @ x
    mean += y.mean().item()
    var += y.pow(2).mean().item()
mean/10000, math.sqrt(var/10000)

(0.0001065013965126127, 1.0001001637333033)

Now we let's re-run our quick and dirty 100-layer network by scaling our weights, which we first choose at random from standard normal distribution inside [-1,1], by $\frac{1}{\sqrt{n}}$, where n is the number of network input connections at a layer, or 512, in our example.

In [24]:
x = torch.randn(512)

for i in range(100):
    a = torch.randn(512,512) * math.sqrt(1./512)
    x = a @ x
x.mean(), x.std()

(tensor(-0.0668), tensor(0.7431))

Success! Our layer outputs didn't explode after 100 of our hypothetical layers. Now while at first glance it may seem like at this point we can call it a day. Real-world neural networks aren't quite as simple as our first example may seem to indicate. For the sake of simplicity, activation functions were omitted. However, we'd never do this in real life, as it's thanks to these non-linear activation functions that come at the end of network layers, that neural nets can do such a great job of approximating the intricate functions that can make impressive predictions using real-world inputs, such as the classification of handwriting samples.

Now up until a few years ago, most commonly used activation functions were symmetric about a given value, and had ranges that asymptotically approached values that were plus/minus a certain distance from this midpoint. The hyperbolic tangent and soft sign functions exemplify this class of activations:

<img src= 'images/softsign_tanh.png' width='300'/>

Image source: [Sefik Ilkin Serengil's blog](https://sefiks.com/2017/11/10/softsign-as-a-neural-networks-activation-function/)

Let's add a hyperbolic tangent activation function after each layer our hypothetical 100-layer network, and then see what happens when we use our above weight initialization scheme.

In [25]:
def tanh(x): return torch.tanh(x)

In [26]:
x = torch.randn(512)

for i in range(100): 
    a = torch.randn(512,512) * math.sqrt(1./512)
    x = tanh(a @ x)
x.mean(), x.std()

(tensor(0.0013), tensor(0.0753))

The standard deviation of activation outputs of the 100th layer is down to about 0.1, but thankfully activations haven't vanished. We should be feeling pretty good about the initialization scheme we just discovered from scratch!

As intuitive as this approach may now seem in retrospect, you may be surprised to hear that this was not, in fact, the conventional approach for initializing weight layers as recently as 2010. When Xavier Glorot and Yoshua Bengio published their landmark paper titled ['Understanding the difficulty of training deep feedforward neural networks'](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf), the "commonly used heuristic" to which they compared their experiments was that of initializing weights from a uniform distribution in [-1,1] and then scaling by $\frac{1}{\sqrt{n}}$.

It turns out this "standard" approach doesn't actually work that well.

In [27]:
x = torch.randn(512)

for i in range(100): 
    a = torch.Tensor(512,512).uniform_(-1, 1) * math.sqrt(1./512)
    x = tanh(a @ x)
x.mean(), x.std()

(tensor(4.2201e-26), tensor(1.0684e-24))

Rerunning our 100-layer tanh network with the "standard" weight initialization caused activation gradients to become infinitessimally small -- they're just about as good as vanished.

This poor performance is what spurred Glorot and Bengio to propose their own weight initialization strategy, which they referred to as "normalized initialization" in their paper, and which is now popularly termed "Xavier initialization."

Xavier initialization initializes a layer's weights with values chosen from a random uniform distribution bounded between $$\pm\frac{\sqrt{6}}{\sqrt{n_{i} + n_{i+1}}}$$ where $n_{i}$ is the number of incoming network connections, or "fan-in," to the layer, and $n_{i+1}$ is the number of outgoing network connections from that layer, also known as the "fan-out."

Glorot and Bengio believed that Xavier weight initialization would  maintain the variance of activations and back-propagated gradients all the way up or down a network. They theorized that this would allow for the successful training of deeper networks, as they observed that Xavier initialization enabled a 5-layer network to  maintain a near identical variance of weight gradients across layers. 

<img src= 'images/xavier_init.png' width='500'/>

Image source: [Glorot & Bengio](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf)

Using "standard" initialization brought about a much bigger gap between weight gradients of the network's lower layers, which were higher, and those of its top-most layers, which were approaching zero.

<img src= 'images/standard_init.png' width='500'/>

Image source: [Glorot & Bengio](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf)

To drive the point home, they demonstrated that networks thus initialized achieved substationally quicker convergence and higher accuracy on the [CIFAR-10 image classification task](https://www.cs.toronto.edu/~kriz/cifar.html).

We can re-run our 100-layer tanh "network" once more, this time using Xavier initialization:

In [28]:
def xavier(m,h): 
    return torch.Tensor(m, h).uniform_(-1, 1)*math.sqrt(6./(m+h))

In [29]:
x = torch.randn(512)

for i in range(100):
    a = xavier(512, 512)
    x = tanh(a @ x)
x.mean(), x.std()

(tensor(-0.0023), tensor(0.0761))

We can see that in our experimental network, Xavier initialization performs pretty identical to our home-grown method that we derived above, where we sampled values from a random normal distribution and scaled by the square root of number of incoming network connections, $n$.

Conceptually, when we are using activation functions that are symmetric about zero and have outputs inside [-1,1], such as softsign and the hyperbolic tangent, it makes sense that we'd want our network layer activation outputs to have a mean of 0 and a standard deviation around 1, on average. And this is precisely what our method and the Xavier method enable.

But what if we're using ReLU activation functions? Would it still make sense to want to scale random initial weight values by the same factor?

<img src= 'images/relu.png' width='300'/>

Image source: [Kanchan Sarkar's blog](https://medium.com/@kanchansarkar/relu-not-a-differentiable-function-why-used-in-gradient-based-optimization-7fef3a4cecec)

Let's use a ReLU activation instead of tanh in one of our hypothetical network's layers and observe that the expected standard deviation of outputs would be.

In [30]:
def relu(x): return x.clamp_min(0.)

In [31]:
mean, var = 0.,0.
for i in range(10000):
    x = torch.randn(512)
    a = torch.randn(512,512)
    y = relu(a @ x)
    mean += y.mean().item()
    var += y.pow(2).mean().item()
mean/10000, math.sqrt(var/10000)

(9.021727816867829, 16.00131549333976)

It turns out that when using a ReLU activation, a single layer will, on average have standard deviation that's very close to the square root of the number of input connections *divided by two*, or $\sqrt{\frac{512}{2}}$ in our example.

In [32]:
math.sqrt(512/2)

16.0

Scaling the values in our weight matrix `a` by this number will give our ReLU layer a standard deviation of 1 on average.

In [33]:
mean, var = 0.,0.
for i in range(10000):
    x = torch.randn(512)
    a = torch.randn(512,512) * math.sqrt(2/512)
    y = relu(a @ x)
    mean += y.mean().item()
    var += y.pow(2).mean().item()
mean/10000, math.sqrt(var/10000)

(0.5642869254142046, 1.0005236392057129)

As we showed before, keeping the standard deviation of layers' activations around 1 will allow us to stack several more layers in a deep neural network without gradients exploding or vanishing.

In fact, this exploration into how to best scale ReLU activations in order to prevent gradients from exploding or vanishing is what motivated Kaiming He et. al. to in 2015 [propose their own initialization scheme](https://arxiv.org/pdf/1502.01852.pdf) that's tailored for deep neural nets that use asymmetric non-linear activations like ReLU.

He et. al. demonstrated that deep networks (e.g a 22-layer CNN) would converge much earlier if the following input weight initialization strategy is employed:

1. Create a tensor with the dimensions appropriate for a weight matrix at a given layer.
2. Populate the tensor with numbers randomly chosen from a standard distribution.
3. Multiply each randomly chosen number by $\sqrt{\frac{2}{n}}$ where $n$ is the number of incoming connections coming into a given layer from the previous layer's output (also known as the "fan-in").
4. Bias tensors are initialized to zero.

In regards to training even deeper networks that used ReLUs, He et. al. found that a 30-layer CNN using Xavier initialization stalled completely and didn't learn at all. However, when the same network was initialized as recommended above it did, in fact, begin to learn and converge.

We can follow the above directions to implement our own version of Kaiming initialization and verify that it can indeed prevent activation outputs from both exploding or vanishing if we use ReLU instead of tanh at all layers of our hypothetical 100-layer network.

In [34]:
def kaiming(m,h): 
    return torch.randn(m,h)*math.sqrt(2./m)

In [35]:
x = torch.randn(512)

for i in range(100): 
    a = kaiming(512, 512)
    x = relu(a @ x)
x.mean(), x.std()

(tensor(0.3847), tensor(0.5676))

Using Kaiming to initialize weights indeed prevents activation outputs from exploding or vanishing, even after 100 linear layers, each with a ReLU activation! 

Let's see what happens if we were to use Xavier initialization, instead.

In [36]:
x = torch.randn(512)

for i in range(100):
    a = xavier(512, 512)
    x = relu(a @ x)
x.mean(), x.std()

(tensor(4.9297e-16), tensor(6.8151e-16))

Ouch! When using Xavier to initialize weights in our hypothetical 100-layer network with ReLUs, activation gradients almost completely vanish!

### Applying Kaiming Init to our MNIST Task

Turning back to the MNIST task, we will first create a simple linear network layer with a ReLU activation function. We'll use Kaiming to  initialize the weights. And as He et. al.'s paper also recommended, we'll initialize the biases to zero.

By the way, here's how to figure out our first layer's weight matrix size: Although the MNIST samples were originally 28x28-sized images, before feeding them into our fully-connected network, we first flattened them into single vectors containing 28 x 28 = 784 pixel values. We've already decided that we want our network's hidden layers to be a size of 50. In order to make this happen, we need to multiply our 784-length vector inputs by a weight matrix that has a height (number of rows) that's also 784 and a width (number of columns) that's 50.

In [37]:
w1 = kaiming(m, nh)
b1 = torch.zeros(nh)
w2 = kaiming(nh,1)
b2 = torch.zeros(1)
w1.mean(), w1.std(), w2.mean(), w2.std()

(tensor(-0.0002), tensor(0.0505), tensor(0.0325), tensor(0.2018))

In [38]:
test_near_zero(w1.mean())
test_near_zero(w1.std()-math.sqrt(2./m))

In [39]:
# Simple linear network layer
def lin(x, w, b): return x@w + b

We can perform a quick test by adding a relu activation to the above layer, plugging in some images into it, and observing the mean and standard deviation of the output values.

In [40]:
t = relu(lin(x_valid, w1, b1))
t.mean(), t.std()

(tensor(0.6102), tensor(0.8934))

In [41]:
#export
from torch.nn import init

Another quick sanity check, just to double check our from-scratch Kaiming init with PyTorch's own Kaiming method:

In [42]:
w1 = torch.zeros(m,nh)
# Choose mode='fan_out' (number of layer outputs), 
# as opposed to 'fan-in' to preserve the variance of 
# weights in the backwards pass.
init.kaiming_normal_(w1, mode='fan_out')
s = relu(lin(x_valid, w1, b1))
s.mean(), s.std()

(tensor(0.5384), tensor(0.8081))

Looks like PyTorch's kaiming method more or less agrees with our homegrown implementation.

### Loss Function: Mean Squared Error (MSE)

MNIST is a multi-class classification problem, and mean squared error as a loss function won't help us get great performance. Categorical cross-entropy  would be a better choice. However, first let's implement MSE to prove to ourselves that we can get a loss function successfully working with our model, period. In a subsequent notebook we'll upgrade to a more ideal substitute.

In [43]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [44]:
%timeit -n 10 _=model(x_valid)

5.27 ms ± 348 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [45]:
assert model(x_valid).shape == torch.Size([x_valid.shape[0],1])

In [46]:
model(x_valid).shape

torch.Size([10000, 1])

In [47]:
#export
def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()

In [48]:
y_train, y_valid = y_train.float(), y_valid.float()

In [49]:
preds = model(x_train)
preds.shape

torch.Size([50000, 1])

In [50]:
mse(preds, y_train)

tensor(20.4509)

### Gradients and Backward Pass

In [51]:
def mse_grad(inp, targ):
    # Gradient of loss with respect to output of previous layer
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1)/inp.shape[0]

In [52]:
def relu_grad(inp, out):
    # Gradient of ReLU with respect to input activations
    inp.g = (inp>0).float() * out.g

In [53]:
def lin_grad(inp, out, w, b):
    # Gradient of matmul (between inputs and weights) with respect 
    # to the inputs.
    inp.g = out.g @ w.t()
    # Weights gradient
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    # Bias gradient
    b.g = out.g.sum(0)

In [54]:
def forward_and_backward(inp, targ):
    # Forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    
    # Backward pass:
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

In [55]:
forward_and_backward(x_train, y_train)

We'll set aside copies of of the gradients produced by our custom methods so that we can benchmark them against the gradients calculated by PyTorch's autograd:

In [56]:
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig = x_train.g.clone()

So to complete our sanity check, let's perform the forward and backward pass once more with PyTorch autograd:

In [57]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [58]:
def forward(inp, targ):
    # Forward pass:
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    return mse(out, targ)

In [59]:
loss = forward(xt2, y_train)

In [60]:
loss.backward()

In [61]:
test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig)

AssertionError: near:
tensor([[ 0.0550, -0.1056,  0.3562,  ...,  0.2126,  0.4059, -0.2055],
        [ 0.0550, -0.1056,  0.3562,  ...,  0.2126,  0.4059, -0.2055],
        [ 0.0550, -0.1056,  0.3562,  ...,  0.2126,  0.4059, -0.2055],
        ...,
        [ 0.0550, -0.1056,  0.3562,  ...,  0.2126,  0.4059, -0.2055],
        [ 0.0550, -0.1056,  0.3562,  ...,  0.2126,  0.4059, -0.2055],
        [ 0.0550, -0.1056,  0.3562,  ...,  0.2126,  0.4059, -0.2055]])
tensor([[ 0.0550, -0.1056,  0.3562,  ...,  0.2126,  0.4059, -0.2055],
        [ 0.0550, -0.1056,  0.3562,  ...,  0.2126,  0.4059, -0.2055],
        [ 0.0550, -0.1056,  0.3562,  ...,  0.2126,  0.4059, -0.2055],
        ...,
        [ 0.0550, -0.1056,  0.3562,  ...,  0.2126,  0.4059, -0.2055],
        [ 0.0550, -0.1056,  0.3562,  ...,  0.2126,  0.4059, -0.2055],
        [ 0.0550, -0.1056,  0.3562,  ...,  0.2126,  0.4059, -0.2055]])

Outstanding! It appears that our home-grown methods can calculate the loss gradients just as well as PyTorch autograd.

## Refactoring Our Model

A theme we'll touch back on repeatedly throughout these notebooks is the importance of refactoring our code into concise, easily readable and interpretable components. 

It's relatively easy to type out several lines of Python that describe a deep learning model or training regime. Many folks have the tendency to then let sit in perpetuity code that should have only been a v.1 rough draft. Taking the time to refactor our deep learning code will make it easier for others to understand both the fine details and general thrust of our code in one sweep. The habit of refactoring also gives us more modular, extensible code that in turn gives us the freedom to more quickly build on and augment our models.

Alternatively put, refactored code is:
* Easier to maintain
* Easier for new contributors/team members to get up to speed on.
* Harder to introduce bugs into.

### Writing Layers as Classes

In [62]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

In [63]:
class Lin():
    def __init__(self, w, b): self.w, self.b = w,b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp @ self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        
        # Note that creating a giant outer-product, just to sum it, 
        # is computationally inefficient.
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

Note that in the `backward()` method in this implementation of the linear layer class we calculate the layer's weight gradient by performing an outer product between a tensor containing a layer's input values for each of the 50,000 training images (with shape `torch.Size([50000, 784])`) and a tensor containing the gradients (that flow in from subsequent layers) on that layer's outputs, for each of the 50,000 training images (with shape `torch.Size([50000, 50])`). 

To perform the outer product, we add an extra final dimension to the first, or inputs, tensor, and also insert a new dimension at the second, or outputs gradient, tensor's second dimension, and then multiply. Summing the result of this multiplication over all 50,000 training images gives us the final tensor containing weight gradients for a given layer after the end of one backward pass.

We will show below that using einstein summation is a more efficient approach, and finally, that performing a matrix multiplication between the transpose of the inputs and the output gradients runs about as fast as einstein summation, while being the most concise implementation:

```
self.w.g = inp.t() @ out.g
```

In [64]:
class MSE():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g  = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [65]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = MSE()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [66]:
# Zero out the gradients.
w1.g, b1.g, w2.g, b2.g = [None]*4
# Uses the weights and biases we initialized above with Kaiming
model = Model(w1, b1, w2, b2)  

In [67]:
%time loss = model(x_train, y_train)

Wall time: 31.9 ms


In [68]:
%time model.backward()

Wall time: 2.26 s


In [69]:
# Make sure our refactored model still calculates gradients properly.
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Adding a Module() class that handles forward() calls for our classes

This allows us to take the `__call__` methods out of the above classes, removing some duplicate code. Also, as we will see just below, the Python code of layer classes that contain intuitively named `forward()`/`backward()` methods is easier to read.

In [70]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception('Not Implemented')
    def backward(self): self.bwd(self.out, *self.args)

In [71]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)
    def bwd(self, out, inp): inp.g = (inp > 0).float() * out.g

In [72]:
class Lin(Module):
    def __init__(self, w, b): self.w, self.b = w,b
        
    def forward(self, inp): return inp @ self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        
        # Einsteim summation is more efficient than the outer 
        # product we previously had implemented.
        self.w.g = torch.einsum('bi,bj->ij', inp, out.g)
        self.b.g = out.g.sum(0)

In [73]:
class MSE(Module):
    # Use the loss function to calculate the loss.
    def forward(self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    # Find the slope of the loss function for a particular set of 
    # inputs and their corresponding targets.
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze() - targ).unsqueeze(-1) / targ.shape[0]

In [74]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = MSE()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
       
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [75]:
w1.g, b1.g, w2.g, b2.g = [None]*4
model = Model()

In [76]:
%time loss = model(x_train, y_train)

Wall time: 24 ms


In [77]:
%time model.backward()

Wall time: 92 ms


In [78]:
6.94/0.141

49.21985815602837

With using einstein summation, our the backward pass is nearly 50 times faster!

In [79]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Rewriting Linear Layer Class without Einsum

In [80]:
class Lin(Module): 
    def __init__(self, w, b): self.w, self.b = w,b
        
    def forward(self, inp): return inp @ self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.t() @ out.g
        self.b.g = out.g.sum(0)

In [81]:
w1.g, b1.g, w2.g, b2.g = [None]*4
model = Model()

In [82]:
%time loss = model(x_train, y_train)

Wall time: 34 ms


In [83]:
%time model.backward()

Wall time: 75 ms


Using PyTorch matmul to compute the weights gradient for our linear layer (`self.w.g = inp.t() @ out.g`) is just as fast as einstein summation but pleasingly more concise. It's always great to not have to program inside of a string (e.g. `'bi,bj->ij'`).

In [84]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Using PyTorch's own classes

Finally, we can run one backward pass using PyTorch's own implementations of the Module class, linear and ReLU layers, and mean squared error loss function, and observe how their performance compares to the classes we wrote from scratch. 

In [85]:
#export
from torch import nn

In [86]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in, nh), nn.ReLU(), nn.Linear(nh, n_out)]
        self.loss = mse
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x.squeeze(), targ)

In [87]:
model = Model(m, nh, 1)

In [88]:
%time loss = model(x_train, y_train)

Wall time: 39 ms


In [89]:
%time loss.backward()

Wall time: 44 ms


In [90]:
146/45.6

3.2017543859649122

PyTorch's own implementations can complete a backward pass three times faster than the best refactoring of our homegrown module, layer, and loss classes.

## Export
We export all the methods we'll need to call in later notebooks.

In [91]:
!./notebook2script_my_reimplementation.py 02_fully_connected_my_reimplementation.ipynb

'.' is not recognized as an internal or external command,
operable program or batch file.
